In [1]:
import tensorflow as tf
# from tensorflow.examples.tutorials.mnist import input_data
# mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

In [2]:
import numpy as np
import pickle

In [3]:
data_files = ['dev', 'test', 'train']

data_path = "./data/%s_dataset.json"
x_file = './op_data1/%s_op_X.pkl'
y_file = './op_data1/%s_op_Y.pkl'
idx_file = './op_data1/%s_op_idx.pkl'

In [4]:
n_input = 1200
n_hidden1 = 512
n_hidden2 = 256
n_hidden3 = 64
n_output = 4

In [5]:
learning_rate = 1e-4
n_iteration = 101
batch_size = 128
dropout = 0.1

In [6]:
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_output])
# keep_prob = tf.placeholder(tf.float32) # used to control the dropout rate

In [7]:
weights = {
    'w1': tf.Variable(tf.truncated_normal([n_input, n_hidden1], stddev=0.1)),
    'w2': tf.Variable(tf.truncated_normal([n_hidden1, n_hidden2], stddev=0.1)),
    'w3': tf.Variable(tf.truncated_normal([n_hidden2, n_hidden3], stddev=0.1)),
    'out': tf.Variable(tf.truncated_normal([n_hidden3, n_output], stddev=0.1))
}

In [8]:
biases = {
    'b1': tf.Variable(tf.constant(0.1, shape=[n_hidden1])),
    'b2': tf.Variable(tf.constant(0.1, shape=[n_hidden2])),
    'b3': tf.Variable(tf.constant(0.1, shape=[n_hidden3])),
    'out': tf.Variable(tf.constant(0.1, shape=[n_output]))
}

In [9]:
layer_1 = tf.add(tf.matmul(X, weights['w1']), biases['b1'])
layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
layer_3 = tf.add(tf.matmul(layer_2, weights['w3']), biases['b3'])
# layer_drop = tf.nn.dropout(layer_3, keep_prob)
output_layer = tf.add(tf.matmul(layer_3, weights['out']), biases['out'])

In [10]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=output_layer))
train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

In [11]:
correct_pred = tf.equal(tf.argmax(output_layer, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [12]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [13]:
def get_data(dt):
    with open(y_file%dt, 'rb') as fy:
        _Y = pickle.load(fy)

    with open(x_file%dt, 'rb') as fx:
        _X = pickle.load(fx)
    return (_X, _Y)

In [14]:
def get_batch(idx, batch_size):
    lw = idx * batch_size
    hs = lw + batch_size
    if hs < len(dev_X):
        return (dev_X[lw:hs][:], dev_Y[lw:hs][:])
    else:
        return (dev_X[lw:][:], dev_Y[lw:][:])

In [15]:
dev_X, dev_Y = get_data('dev')

In [16]:
n_batches = len(dev_X)//batch_size

In [17]:
len(dev_X)

8288

In [18]:
# train on mini batches
for i in range(n_iteration):
    for j in range(n_batches+1):
        (batch_x, batch_y) = get_batch(j, batch_size)
        sess.run(train_step, feed_dict={X: batch_x, Y: batch_y})
        
        # print loss and accuracy (per minibatch)
    if i % 5 == 0:
        minibatch_loss, minibatch_accuracy = sess.run([cross_entropy, accuracy], feed_dict={X: batch_x, Y: batch_y})
        print("Iteration", str(i), "\t| Batch", str(j), "\t| Loss =", str(minibatch_loss), "\t| Accuracy =", str(minibatch_accuracy))

Iteration 0 	| Batch 64 	| Loss = 0.44177935 	| Accuracy = 0.84375
Iteration 5 	| Batch 64 	| Loss = 0.25247836 	| Accuracy = 0.9375
Iteration 10 	| Batch 64 	| Loss = 0.23269363 	| Accuracy = 0.9375
Iteration 15 	| Batch 64 	| Loss = 0.22100979 	| Accuracy = 0.9270833
Iteration 20 	| Batch 64 	| Loss = 0.21252207 	| Accuracy = 0.9479167
Iteration 25 	| Batch 64 	| Loss = 0.20639189 	| Accuracy = 0.9479167
Iteration 30 	| Batch 64 	| Loss = 0.20199807 	| Accuracy = 0.9479167
Iteration 35 	| Batch 64 	| Loss = 0.19880314 	| Accuracy = 0.9479167
Iteration 40 	| Batch 64 	| Loss = 0.19637878 	| Accuracy = 0.9479167
Iteration 45 	| Batch 64 	| Loss = 0.19445612 	| Accuracy = 0.9479167
Iteration 50 	| Batch 64 	| Loss = 0.1928755 	| Accuracy = 0.9479167
Iteration 55 	| Batch 64 	| Loss = 0.19154048 	| Accuracy = 0.9479167
Iteration 60 	| Batch 64 	| Loss = 0.19038735 	| Accuracy = 0.9479167
Iteration 65 	| Batch 64 	| Loss = 0.18937276 	| Accuracy = 0.9479167
Iteration 70 	| Batch 64 	| Los

In [19]:
test_X, test_Y = get_data('train')

test_accuracy = sess.run(accuracy, feed_dict={X: test_X, Y: test_Y})
print("\nAccuracy on test set : ", test_accuracy)


Accuracy on test set :  0.88998
